In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from lightning.pytorch.loggers import TensorBoardLogger
import sys

from lightning.pytorch.callbacks import StochasticWeightAveraging
import glob
import os
import lightning as L
sys.path.insert(0, "/home/tordjx/DATA_DIR/config/projects/TRANSFERABLECLEANDATAPOISONING/lib/python/")
from customlib.dataloaders import CustomDataset
data_dir = '/home/tordjx/DATA_DIR/managed_folders/TRANSFERABLECLEANDATAPOISONING/LMc8Smw6/'
import timm
import numpy as np
import pandas as pd
import skimage.io
import matplotlib.pyplot as plt
import torchmetrics
import torch
torch.manual_seed(0)
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Resize
train_dataset = CustomDataset(data_dir, train=True, poison_dir = "/home/tordjx/DATA_DIR/managed_folders/TRANSFERABLECLEANDATAPOISONING/BqfvFGr8")
test_dataset = CustomDataset(data_dir, train=False)

# Create data loaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers = 19  )
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers = 19)


class ClassifierModel(L.LightningModule):
    def __init__(self,encoder_name ,intermediate_size = 64, nclasses=43, lr = 5e-4, encoder_freeze = True):
        super().__init__()
        self.encoder_name  = encoder_name
        self.encoder = timm.create_model(encoder_name, pretrained = True, num_classes=0)
        self.encoder_freeze = encoder_freeze
        if self.encoder_freeze : 
            for param in self.encoder.parameters():
                param.requires_grad = False
        feature_size = (self.encoder(torch.rand(2,3,128,128))).shape[-1]
        self.decoder = nn.Linear(feature_size, intermediate_size)
        self.classif = nn.Linear(intermediate_size, nclasses)
        self.criterion = nn.CrossEntropyLoss()
        self.lr = lr
        self.metric = torchmetrics.Accuracy(task="multiclass", num_classes=nclasses)
    def training_step(self, batch, batch_idx):
        x, y = batch
        x_hat = self.forward(x)
        loss = self.criterion(x_hat, y)
        accuracy = self.metric(x_hat, y)
        self.log("train_loss", loss)
        self.log('train_accuracy', accuracy)
        return loss
    def validation_step(self, batch, batch_idx):
        x, y = batch
        x_hat = self.forward(x)
        loss = self.criterion(x_hat,y)
        accuracy = self.metric(x_hat, y)
        self.log("val_loss", loss)
        self.log('val_accuracy', accuracy)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay = 1e-5)
        return optimizer
    def forward(self, x):
        if self.encoder_freeze : 
            with torch.no_grad():
                x = self.encoder(x)
        x = nn.functional.relu(self.decoder(x))
        x = nn.functional.relu(self.classif(x))
        return x


In [2]:
encoder_name = "resnet34"
os.environ["TENSORBOARD_BINARY"]="/home/tordjx/DATA_DIR/code-envs/python/rapids/bin/tensorboard"
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/
#CLEAN
train_dataset = CustomDataset(data_dir, train=True)#,poison_dir = "/home/tordjx/DATA_DIR/managed_folders/TRANSFERABLECLEANDATAPOISONING/BqfvFGr8")
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers = 19  )
encoder_freeze = True
is_poisonned = "poison" if train_dataset.poison_dir != None else "sans poison"
log_name = encoder_name + " , "+ is_poisonned+ "" 
logger = TensorBoardLogger("lightning_logs/", name=log_name)
trainer = L.Trainer(max_epochs=5,val_check_interval = 1.0, devices = 1,logger=logger)
classifier = ClassifierModel(encoder_name = encoder_name, encoder_freeze = encoder_freeze)
trainer.fit(model=classifier, train_dataloaders=train_loader, val_dataloaders = test_loader)
#10%
train_dataset = CustomDataset(data_dir, train=True, poison_num = 4000,poison_dir = "/home/tordjx/DATA_DIR/managed_folders/TRANSFERABLECLEANDATAPOISONING/BqfvFGr8")
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers = 19  )
encoder_freeze = True
is_poisonned = "poison" if train_dataset.poison_dir != None else "sans poison"
log_name = encoder_name + " , "+ is_poisonned+ " , 4000 poisons" 
logger = TensorBoardLogger("lightning_logs/", name=log_name)
trainer = L.Trainer(max_epochs=5,val_check_interval = 1.0, devices = 1,logger=logger)
classifier = ClassifierModel(encoder_name = encoder_name, encoder_freeze = encoder_freeze)
trainer.fit(model=classifier, train_dataloaders=train_loader, val_dataloaders = test_loader)
#50%
train_dataset = CustomDataset(data_dir, train=True, poison_num = 20000,poison_dir = "/home/tordjx/DATA_DIR/managed_folders/TRANSFERABLECLEANDATAPOISONING/BqfvFGr8")
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers = 19  )
encoder_freeze = True
is_poisonned = "poison" if train_dataset.poison_dir != None else "sans poison"
log_name = encoder_name + " , "+ is_poisonned+ " ,  20000 poisons" 
logger = TensorBoardLogger("lightning_logs/", name=log_name)
trainer = L.Trainer(max_epochs=5,val_check_interval = 1.0, devices = 1,logger=logger)
classifier = ClassifierModel(encoder_name = encoder_name, encoder_freeze = encoder_freeze)
trainer.fit(model=classifier, train_dataloaders=train_loader, val_dataloaders = test_loader)
#100%
train_dataset = CustomDataset(data_dir, train=True, poison_dir = "/home/tordjx/DATA_DIR/managed_folders/TRANSFERABLECLEANDATAPOISONING/BqfvFGr8")
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers = 19  )
encoder_freeze = True
is_poisonned = "poison" if train_dataset.poison_dir != None else "sans poison"
log_name = encoder_name + " , "+ is_poisonned+ "" 
logger = TensorBoardLogger("lightning_logs/", name=log_name)
trainer = L.Trainer(max_epochs=5,val_check_interval = 1.0, devices = 1,logger=logger)
classifier = ClassifierModel(encoder_name = encoder_name, encoder_freeze = encoder_freeze)
trainer.fit(model=classifier, train_dataloaders=train_loader, val_dataloaders = test_loader)

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2024-01-06 20:44:40.829684: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-06 20:44:40.830889: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-06 20:44:40.848107: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-06 20:44:40.848134: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-06 20:44:40.848147: E tensorflow/compiler/xla/stream_executor/cuda

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
Missing logger folder: lightning_logs/resnet34 , sans poison
2024-01-06 20:44:43.763834: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-06 20:44:43.765014: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-06 20:44:43.781142: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-06 20:44:43.781158: E tensorfl

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [3]:
os.environ["TENSORBOARD_BINARY"]="/home/tordjx/DATA_DIR/code-envs/python/rapids/bin/tensorboard"
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

In [3]:
encoder_name = "tiny_vit_21m_224.dist_in22k_ft_in1k"
os.environ["TENSORBOARD_BINARY"]="/home/tordjx/DATA_DIR/code-envs/python/rapids/bin/tensorboard"
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/
#CLEAN
train_dataset = CustomDataset(data_dir, train=True)#,poison_dir = "/home/tordjx/DATA_DIR/managed_folders/TRANSFERABLECLEANDATAPOISONING/BqfvFGr8")
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers = 19  )
encoder_freeze = True
is_poisonned = "poison" if train_dataset.poison_dir != None else "sans poison"
log_name = encoder_name + " , "+ is_poisonned+ " 5 backbones, white box" 
logger = TensorBoardLogger("lightning_logs/", name=log_name)
trainer = L.Trainer(max_epochs=5,val_check_interval = 1.0, devices = 1,logger=logger)
classifier = ClassifierModel(encoder_name = encoder_name, encoder_freeze = encoder_freeze)
trainer.fit(model=classifier, train_dataloaders=train_loader, val_dataloaders = test_loader)
#10%
train_dataset = CustomDataset(data_dir, train=True, poison_num = 4000,poison_dir = "/home/tordjx/DATA_DIR/managed_folders/TRANSFERABLECLEANDATAPOISONING/BqfvFGr8")
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers = 19  )
encoder_freeze = True
is_poisonned = "poison" if train_dataset.poison_dir != None else "sans poison"
log_name = encoder_name + " , "+ is_poisonned+ " 5 backbones, white box, 4000 poisons" 
logger = TensorBoardLogger("lightning_logs/", name=log_name)
trainer = L.Trainer(max_epochs=5,val_check_interval = 1.0, devices = 1,logger=logger)
classifier = ClassifierModel(encoder_name = encoder_name, encoder_freeze = encoder_freeze)
trainer.fit(model=classifier, train_dataloaders=train_loader, val_dataloaders = test_loader)
#50%
train_dataset = CustomDataset(data_dir, train=True, poison_num = 20000,poison_dir = "/home/tordjx/DATA_DIR/managed_folders/TRANSFERABLECLEANDATAPOISONING/BqfvFGr8")
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers = 19  )
encoder_freeze = True
is_poisonned = "poison" if train_dataset.poison_dir != None else "sans poison"
log_name = encoder_name + " , "+ is_poisonned+ " 5 backbones, white box, 20000 poisons" 
logger = TensorBoardLogger("lightning_logs/", name=log_name)
trainer = L.Trainer(max_epochs=5,val_check_interval = 1.0, devices = 1,logger=logger)
classifier = ClassifierModel(encoder_name = encoder_name, encoder_freeze = encoder_freeze)
trainer.fit(model=classifier, train_dataloaders=train_loader, val_dataloaders = test_loader)
#100%
train_dataset = CustomDataset(data_dir, train=True, poison_dir = "/home/tordjx/DATA_DIR/managed_folders/TRANSFERABLECLEANDATAPOISONING/BqfvFGr8")
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers = 19  )
encoder_freeze = True
is_poisonned = "poison" if train_dataset.poison_dir != None else "sans poison"
log_name = encoder_name + " , "+ is_poisonned+ " 5 backbones, white box" 
logger = TensorBoardLogger("lightning_logs/", name=log_name)
trainer = L.Trainer(max_epochs=5,val_check_interval = 1.0, devices = 1,logger=logger)
classifier = ClassifierModel(encoder_name = encoder_name, encoder_freeze = encoder_freeze)
trainer.fit(model=classifier, train_dataloaders=train_loader, val_dataloaders = test_loader)

Reusing TensorBoard on port 6006 (pid 491720), started 0:47:50 ago. (Use '!kill 491720' to kill it.)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


model.safetensors:   0%|          | 0.00/84.9M [00:00<?, ?B/s]

Missing logger folder: lightning_logs/tiny_vit_21m_224.dist_in22k_ft_in1k , sans poison 5 backbones, white box
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name      | Type               | Params
-------------------------------------------------
0 | encoder   | TinyVit            | 20.6 M
1 | decoder   | Linear             | 36.9 K
2 | classif   | Linear             | 2.8 K 
3 | criterion | CrossEntropyLoss   | 0     
4 | metric    | MulticlassAccuracy | 0     
-------------------------------------------------
39.7 K    Trainable params
20.6 M    Non-trainable params
20.7 M    Total params
82.645    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [4]:
image,y = next(iter(train_loader))
with torch.no_grad() : 
    perturb = gan.generator(image.cuda()).cpu() 

for idx in range(3):
    
    fig, ax = plt.subplots(1,6, figsize = (30,5))
    ax[0].imshow(perturb[idx].moveaxis(0,-1))
    ax[0].set_title("Full perturbation")
    ax[1].imshow(perturb[idx][0])
    ax[1].set_title("Red perturbation")
    ax[2].imshow(perturb[idx][1])
    ax[2].set_title("Green perturbation")
    ax[3].imshow(perturb[idx][2])
    ax[3].set_title("Blue perturbation")
    ax[4].imshow(image[idx].moveaxis(0,-1))
    ax[4].set_title("Clean image")
    ax[5].imshow(perturb[idx].moveaxis(0,-1)+image[idx].moveaxis(0,-1))
    ax[5].set_title("Image + perturbation")
    plt.show()

NameError: name 'gan' is not defined